# **Automated Machine Learning**

---

### **Torch Reptile - Parallel Metalearning**
*Fall 2020 | Ruduan B.F. Plug*

---

<font size="1">*Based on the Original Implementation by Alex Nichol & John Schulman [[1]](https://openai.com/blog/reptile/)*</font>

### Meta Libraries

In [17]:
# System Utility
import sys

# IPython Notebook Utilities
from IPython.display import clear_output
import tqdm.notebook as tqdm
clear_output()

print(sys.version)

3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]


### Packages

In [18]:
# Data Processing
import numpy as np
import pandas as pd

# Model Library
import tensorflow as tf

# Parallel Compute
import torch
import torch.nn as nn

# Data Visualization
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter

# Utility Libraries
import random
import math
from time import time
from copy import deepcopy
from datetime import datetime

# Initialize Device
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print("Torch Version\t", torch.__version__)

Torch Version	 2.9.0+cpu


### Configuration

In [19]:
data_folder = "data"
np.random.seed(int(time()))
torch.manual_seed(int(time()))

### Reptile TensorFlow

#### Class Definition

In [20]:
class Reptile:

  def __init__(self, model, log, params):

    # Intialize Reptile Parameters
    self.inner_step_size = params[0]
    self.inner_batch_size = params[1]
    self.outer_step_size = params[2]
    self.outer_iterations = params[3]
    self.meta_batch_size = params[4] 
    self.eval_iterations = params[5] 
    self.eval_batch_size = params[6]

    # Initialize Torch Model and Tensorboard
    self.model = model.to(device)
    self.log = log

  def reset(self):

    # Reset Training Gradients
    self.model.zero_grad()
    self.current_loss = 0
    self.current_batch = 0

  def train(self, task):

    # Train from Scratch
    self.reset()

    # Outer Training Loop
    for outer_iteration in tqdm.tqdm(range(self.outer_iterations)):

      # Track Current Weights
      current_weights = deepcopy(self.model.state_dict())

      # Inner Training Loop
      for inner_iteration in range(self.inner_batch_size):

          # Sample a new Subtask
          x_task, y_task = sample(task)

          batch_loss = self.loss(x_task, y_task)
          batch_loss.backward()

          # Update Model Parameters
          for theta in self.model.parameters():

            # Get Parameter Gradient
            grad = theta.grad.data

            # Update Model Parameter
            theta.data -= self.inner_step_size * grad

          # Update Model Loss from Torch Model Tensor
          loss_tensor = batch_loss.cpu()
          self.current_loss += loss_tensor.item()
          self.current_batch += 1

      # Linear Cooling Schedule
      alpha = self.outer_step_size * (1 - outer_iteration / self.outer_iterations)

      # Get Current Candidate Weights
      candidate_weights = self.model.state_dict()

      # Transfer Candidate Weights to Model State Checkpoint
      state_dict = {candidate: (current_weights[candidate] + alpha * 
                               (candidate_weights[candidate] - current_weights[candidate])) 
                                for candidate in candidate_weights}
      self.model.load_state_dict(state_dict)
      
      # Log new Training Loss
      self.log.add_scalar('Model Estimate/Loss',
                           self.current_loss / self.current_batch,
                           outer_iteration)

  def loss(self, x, y):

    # Reset Torch Gradient
    self.model.zero_grad()

    # Calculate Torch Tensors
    x = torch.tensor(x, device = device, dtype = torch.float32)
    y = torch.tensor(y, device = device, dtype = torch.float32)

    # Estimate over Sample
    yhat = self.model(x)

    # Regression Loss over Estimate
    loss = nn.MSELoss()
    output = loss(yhat, y)

    return output

  def predict(self, x):

    # Estimate using Torch Model
    t = torch.tensor(x, device = device, dtype = torch.float32)
    t = self.model(t)

    # Bring Torch Tensor from GPU to System Host Memory
    t = t.cpu()

    # Return Estimate as Numpy Float
    y = t.data.numpy()

    return y

  def eval(self, X_eval, y_eval, gradient_steps=5):

    # Sample Points from Task Sample Space
    x = torch.tensor(X_eval, device=device, dtype=torch.float32)
    y = torch.tensor(np.array(y_eval), device=device, dtype=torch.float32)

    # Store Meta-Initialization Weights
    meta_weights = deepcopy(self.model.state_dict())

    # Get Estimate Loss over Meta-Initialization
    loss_t = self.loss(X_eval,y_eval).cpu()
    meta_loss = loss_t.item()

    # Calculcate Estimate over Gradient Steps
    for step in range(gradient_steps):

      # Calculate Evaluation Loss and Backpropagate
      eval_loss = self.loss(X_eval,y_eval)
      eval_loss.backward()

      # Update Model Estimate Parameters
      for theta in self.model.parameters():

        # Get Parameter Gradient
        grad = theta.grad.data

        # Update Model Parameter
        theta.data -= self.inner_step_size * grad

    # Get Estimate Loss over Evaluation
    loss_t = self.loss(x,y).cpu()
    estimate_loss = loss_t.item()
    evaluation_loss = abs(meta_loss - estimate_loss)/len(X_eval)
    
    # Restore Meta-Initialization Weights
    self.model.load_state_dict(meta_weights)

    preds = self.predict(X_eval)
    preds = (preds > 0.5).astype(float)
    accuracy = np.mean(preds.squeeze() == np.array(y_eval).astype(float))

    return accuracy, evaluation_loss

#### PyTorch Module

In [21]:
class TorchModule(nn.Module):

  def __init__(self, in_features, n):

    # Initialize PyTorch Base Module
    super(TorchModule, self).__init__()

    # Define Multi-Layer Perceptron
    self.input = nn.Linear(in_features,n)
    self.hidden_in = nn.Linear(n,n)
    self.hidden_out = nn.Linear(n,n)
    self.output = nn.Linear(n,1)

  def forward(self, x):

    # PyTorch Feed Forward Subroutine
    x = torch.tanh(self.input(x))
    x = torch.tanh(self.hidden_in(x))
    x = torch.tanh(self.hidden_out(x))
    y = self.output(x)

    return y

### Learning Task

#### Task Sampler

In [22]:
def sample(task):

  normal_idx = np.where(y_train == 0)[0]
  attack_idx = np.where(y_train == 1)[0]
  half = 64

  idx = np.concatenate([
      np.random.choice(normal_idx, half, replace=False),
      np.random.choice(attack_idx, half, replace=False)
  ])

  np.random.shuffle(idx)
  y_batch = y_train.values[idx] if hasattr(y_train, "values") else y_train[idx]
  y_batch = y_batch.reshape(-1, 1)

  return X_train[idx], y_batch

## Dataset

In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the preprocessed dataset
df = pd.read_csv("dataset/preprocessed_DNN.csv")
print("Loaded preprocessed dataset:", df.shape)

# Separate features and labels
X = df.drop(columns=["Attack_label", "Attack_type"], errors="ignore")
y = df["Attack_label"].astype(int)

# Scale features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split for training and testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

Loaded preprocessed dataset: (1909671, 97)


## Experiments

In [24]:
# Define Experiment Parameters
inner_step_size = 0.02
inner_batch_size = 16

outer_step_size = 0.1
outer_iterations = 100
meta_batch_size = 32

eval_iterations = 32
eval_batch_size = 10
eval_range = range(1,11)

model_size = 32
sample_radius = 20
sample_count = 100

params = [inner_step_size, inner_batch_size,
          outer_step_size, outer_iterations, meta_batch_size,
          eval_iterations, eval_batch_size]

# Define Experiment Task and Model
log = SummaryWriter(data_folder)
model = Reptile(TorchModule(X_train.shape[1], model_size), log, params)

# Train Model
model.train(task=None)

with torch.no_grad():
  preds = model.predict(X_test)
  preds = (preds > 0.5).astype(float)
  acc = np.mean(preds.squeeze() == y_test.values.astype(float))
  print(f"Test Accuracy: {acc:.4f}")
  log.add_scalar('Model Estimate/Test_Accuracy', acc)

log.close()

  0%|          | 0/100 [00:00<?, ?it/s]

Test Accuracy: 0.9974


### Results

In [25]:
%load_ext tensorboard
%reload_ext tensorboard
%tensorboard --logdir data